In [1]:
import torch
from transformers import Data2VecTextForMultipleChoice, AutoTokenizer
from PIL import Image
import requests

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
version = "facebook/data2vec-text-base"
prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
choice0 = "It is eaten with a fork and a knife."
choice1 = "It is eaten while held in the hand."

# AutoTokenizer

In [4]:
tokenizer: AutoTokenizer = AutoTokenizer.from_pretrained(version)
tokenizer

RobertaTokenizerFast(name_or_path='facebook/data2vec-text-base', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)}, clean_up_tokenization_spaces=True)

## special ids and tokens

In [5]:
print(tokenizer.all_special_ids)
print(tokenizer.all_special_tokens)

[0, 2, 3, 1, 50264]
['<s>', '</s>', '<unk>', '<pad>', '<mask>']


In [6]:
tokenizer.mask_token_id

50264

## processor

In [20]:
inputs = tokenizer(
    [prompt, prompt],
    [choice0, choice1],
    return_tensors = "pt",      # 返回数据格式 np pt tf jax
    padding = True,             # 填充方式选择 [True, 'longest', 'max_length', 'do_not_pad']
    # max_length = max_length,  # 如果使用max_length要将padding设置为 "max_length"
    add_special_tokens = True,  # text添加特殊key
).to(device, torch.float16)
inputs

{'input_ids': tensor([[    0,  1121,  2627,     6,  9366,  1665,    11,  4828,  9629,     6,
           215,    25,    23,    10,  2391,     6,    16,  2633,  9977,  5895,
           196,     4,     2,     2,   243,    16, 18804,    19,    10, 20935,
             8,    10,  7023,     4,     2],
        [    0,  1121,  2627,     6,  9366,  1665,    11,  4828,  9629,     6,
           215,    25,    23,    10,  2391,     6,    16,  2633,  9977,  5895,
           196,     4,     2,     2,   243,    16, 18804,   150,   547,    11,
             5,   865,     4,     2,     1]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]], device='cuda:0')}

In [26]:
inputs['input_ids']

tensor([[    0,  1121,  2627,     6,  9366,  1665,    11,  4828,  9629,     6,
           215,    25,    23,    10,  2391,     6,    16,  2633,  9977,  5895,
           196,     4,     2,     2,   243,    16, 18804,    19,    10, 20935,
             8,    10,  7023,     4,     2],
        [    0,  1121,  2627,     6,  9366,  1665,    11,  4828,  9629,     6,
           215,    25,    23,    10,  2391,     6,    16,  2633,  9977,  5895,
           196,     4,     2,     2,   243,    16, 18804,   150,   547,    11,
             5,   865,     4,     2,     1]], device='cuda:0')

# Data2VecTextForMultipleChoice

Data2VecText Model with a multiple choice classification head on top (a linear layer on top of the pooled output and a softmax) e.g. for RocStories/SWAG tasks.

In [17]:
model: Data2VecTextForMultipleChoice = Data2VecTextForMultipleChoice.from_pretrained(version, torch_dtype=torch.float16).to(device)
model

Some weights of Data2VecTextForMultipleChoice were not initialized from the model checkpoint at facebook/data2vec-text-base and are newly initialized: ['data2vec_text.pooler.dense.weight', 'data2vec_text.pooler.dense.bias', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Data2VecTextForMultipleChoice(
  (data2vec_text): Data2VecTextModel(
    (embeddings): Data2VecTextForTextEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): Data2VecTextEncoder(
      (layer): ModuleList(
        (0-11): 12 x Data2VecTextLayer(
          (attention): Data2VecTextAttention(
            (self): Data2VecTextSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): Data2VecTextSelfOutput(
              (dense): Linear(in_features=76

In [27]:
model.eval()
with torch.inference_mode():
    outputs = model(**{k: v.unsqueeze(0) for k, v in inputs.items()})
outputs

MultipleChoiceModelOutput(loss=None, logits=tensor([[-0.0390, -0.0401]], device='cuda:0'), hidden_states=None, attentions=None)

In [29]:
logits = outputs.logits
print(logits.shape)
print(logits)

torch.Size([1, 2])
tensor([[-0.0390, -0.0401]], device='cuda:0')


In [31]:
logits.softmax(dim=-1)

tensor([[0.5003, 0.4997]], device='cuda:0')